In [1]:
import getexifdata
from PIL import Image, ExifTags
import glob, os
import json
import io

IMG_SM = 400

# process every dir in Places/
for base_path in glob.glob("../Places/*/"):
    
    print("processing place: " + base_path)

    ####################
    ###
    ### helper functions
    ###
    
    def strip_dir(fpath, dir):
        i = fpath.index(dir)+len(dir)
        return fpath[i:len(fpath)]

    def get_fname(fpath):
        return fpath.split("/")[-1]

    def get_flabel(fpath, ext):
        return get_fname(fpath).replace(ext, "")

    def find_audio_for_img(label):
        for fpath in glob.glob(base_path + "aud/*.mp3"):
            flabel = get_flabel(fpath, ".mp3")
            if flabel == label:
                return strip_dir(fpath, "/aud/")
        return None # no warning needed
    
    def save_err_img(fpath, msg):
        print(msg + " : " + fpath)
        im = Image.open(fpath)
        im.save(base_path + "imgErr/" + get_fname(fpath))
    
    def get_gps_for_fpath(fpath):
        im = Image.open(fpath)
        edat = getexifdata.get_exif_data(im)
        lat, lng = getexifdata.get_lat_lon(edat)
        if lat == None or lng == None:
            save_err_img(fpath, "WARNING! no gps data for image")
            return None
        else:
            return {"lat":lat, "lng":lng}

    try:
        to_unicode = unicode
    except NameError:
        to_unicode = str    
    
    def find_kml():
        return [strip_dir(fpath, "/kml/") for fpath in glob.glob(base_path + "kml/*.kml")]
    
    def reorient_img(fileName, height):
        # thanks storm_to : http://stackoverflow.com/questions/4228530/pil-thumbnail-is-rotating-my-image 
        fpath = base_path + "img/" + fileName
        image=Image.open(fpath)
        
        try:
            for orientation in ExifTags.TAGS.keys() : 
                if ExifTags.TAGS[orientation]=='Orientation' : break 
            exif=dict(image._getexif().items())

            if   exif[orientation] == 3 : 
                image=image.rotate(180, expand=True)
            elif exif[orientation] == 6 : 
                image=image.rotate(270, expand=True)
            elif exif[orientation] == 8 : 
                image=image.rotate(90, expand=True)
        except:
            save_err_img(fpath, "WARNING! no EXIF data for image")
        
        # thumnail
        r = float(height) / image.size[1]
        w = float(image.size[0]) * r
        image.thumbnail((w, height), Image.ANTIALIAS)
        image.save(base_path + "imgSm/" + fileName)

    ####################
    ###
    ### sanity check
    ###

    # sanity check: every audio file should have a matching img
    def ensure_audio_img_match():
        # error if an audio file has no image. slow, but we should ensure this.
        for fpath in glob.glob(base_path + "aud/*.mp3"):
            match = False
            for imgpath in glob.glob(base_path + "img/*.jpg"):
                if get_flabel(imgpath, ".jpg") == get_flabel(fpath, ".mp3"):
                    match = True
            if not match:
                #raise FileNotFoundError("no image for audio file: " + fpath)
                return True

    # sanity check: every image should have GPS coordinates
    def ensure_img_gps():
        [get_gps_for_fpath(fpath) for fpath in glob.glob(base_path + "img/*.jpg")]

    ####################
    ###
    ### run
    ###

    ensure_img_gps()
    ensure_audio_img_match()

    # read json template
    with open(base_path + "info_template.json") as data_file:
        data = json.load(data_file)
        data["layers"] = find_kml()

        # for every image...
        for fpath in glob.glob(base_path + "img/*.jpg"):
            
            # compile location info
            flabel = get_flabel(fpath, ".jpg")
            fnam = get_fname(fpath)
            
            #print("processing: " + fnam)
            
            marker = {
                "label": flabel,
                "loc": get_gps_for_fpath(fpath),
                "img": fnam,
                "aud": find_audio_for_img(flabel)
            }
            data["locations"].append(marker)
            
            # save web-friendly image (rotated & small)
            reorient_img(fnam, IMG_SM)
        
        # Write JSON file
        # http://stackoverflow.com/questions/12309269/how-do-i-write-json-data-to-a-file-in-python
        with io.open(base_path + "info.json", 'w', encoding='utf8') as outfile:
            str_ = json.dumps(data,
                              indent=4, sort_keys=True,
                              separators=(',', ':'), ensure_ascii=False)
            outfile.write(to_unicode(str_))
        
        print("done.")


processing place: ../Places/CostaRica/
WARNING! no gps data for image : ../Places/CostaRica/img/2017-01-05 10.14.51.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/2017-01-05 10.37.50 HDR.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/2017-01-09 13.15.18 HDR.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/2017-01-09 13.15.32.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/2017-01-09 13.24.22-2.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/2017-01-09 14.16.45.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/2017-01-10 16.37.07.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/cabo blanco 0.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/cabo blanco 2.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/cabo blanco 4.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/curu 0.jpg
WARNING! no gps data for image : ../Places/CostaRica/img/la barra honda 0.jpg
WARNING! no g